In [41]:
import torch
from torch_geometric.data import Data, DataLoader

In [19]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1, 2], [0, 0], [1, -2]], dtype=torch.float)


In [20]:
data = Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 4], x=[3, 2])

In [25]:
from torch_geometric.datasets import KarateClub
dataset = KarateClub()
dataset

KarateClub()

In [38]:
dataset[0]

Data(edge_index=[2, 156], train_mask=[34], x=[34, 34], y=[34])

In [43]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='./tmp', use_node_attr=True, name='ENZYMES')

Extracting tmp\ENZYMES\ENZYMES.zip
Processing...
Done!


In [44]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [45]:
batch = next(iter(loader))

In [46]:
batch

Batch(batch=[916], edge_index=[2, 3654], x=[916, 21], y=[32])

In [50]:
batch.batch[:10]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [51]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Processing...
Done!


Cora()

In [53]:
data = dataset[0]
data.x

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [57]:
dataset.num_node_features

1433

In [65]:
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [64]:
model = Net()
data = dataset[0]
predictions = model(data)
predictions.shape

torch.Size([2708, 7])

In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(10):
    predictions = model(data)
    loss = criterion(predictions[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f'{epoch=} loss={loss.item()}')

epoch=0 loss=1.9465255737304688
epoch=1 loss=1.8578877449035645
epoch=2 loss=1.727400779724121
epoch=3 loss=1.6284301280975342
epoch=4 loss=1.4633522033691406
epoch=5 loss=1.3402299880981445
epoch=6 loss=1.2316430807113647
epoch=7 loss=1.1141530275344849
epoch=8 loss=1.0238696336746216
epoch=9 loss=0.9351387023925781


In [68]:
model.eval()
_, predictions = model(data).max(dim=1)
correct = int(predictions[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7350
